In the first question, I have suggested that we don't really know the number of clusters as increasing K caused more clusters. In this question, I seek answer to how many clusters are there given the data and its properties. 

In [1]:
from __future__ import (absolute_import, print_function, division)
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
import os
from scipy.stats import dirichlet
import numpy as np
from scipy.stats import wishart
from scipy.stats import norm
# import numba
from scipy.stats import multivariate_normal as mvn
from numpy.linalg import inv
from numpy import log as ln
from scipy.special import psi
from numpy.random import random as rand
%matplotlib 

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days
Vendor:  Continuum Analytics, Inc.


Using matplotlib backend: MacOSX


Package: mkl
Message: trial mode expires in 28 days


Mixture model used to generate data:

* Model: $c_i ≈ Discrete(\pi)$, $x_i ≈ p(x|\theta_{c_i})$

* Priors: $\pi ≈ Dirichlet( \alpha, \alpha, ..., \alpha)$, $\theta_j ≈ p(\theta)$

Gaussian Mixture Model:
$\theta_j = \{ \mu_j, \Lambda_j\}$

$p(x|\theta_j) = Normal(\mu_j, \Lambda_j^{-1})$ and $p(\theta) = Normal-Wishart

**The Dirichlet process "uncovers" the correct number of clusters. Marginalization of $\pi$ will save the day in Gibbs sampling setup.**

Overall Gibbs Sampling Procedure
-----
1. For each iteration:

    a. Sample each $c_i$ given the most recent $\pi, \theta, and\ other\ c_{-i}$
    
    b. Sample each $\theta_i$ given the most recent c, $\pi$, and $\theta_{-i}$ 
    
    c. Sample $\pi$ given the most recent c and $\theta$
    
**note**: just like the earlier model in problem 2, 


In [3]:
def sample_c_i():
    """Indicates how likely each cluster is for the data point in x_i and pre-weight.
    Returns
    ------
    A normalized K dimensional vector. This is to be used as parameter of Discrete distro 
    to sample a new index value for c_i.    
    """
    pass

In [ ]:
def sample_theta_j():
    """Only depends ont he data in x_train that is assigned to cluster j"""
    pass

In [2]:
os.chdir('/Users/arkilic/Desktop/')

# Step 1

x_train = pd.read_csv('data.txt', header=None)
n, d = x_train.shape